In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import plotly 
import chart_studio.plotly.plotly as py
import chart_studio
import plotly.express as px
import plotly.graph_objs as go
from matplotlib.pyplot import figure
from datetime import datetime
import pandas_ta as ta


from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import re

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string


from wordcloud import WordCloud

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec


#import plotly.graph_objects as go

In [2]:
# Loading the dataset to a dataframe
#sentence_file = "C:\Users\Dennis\Desktop\Coding\Final Project\Data\Combined_News_DJIA"
df = pd.read_csv(r'C:\Users\Dennis\Desktop\Coding\Final Project\Data\Combined_News_DJIA.csv', parse_dates=[1])

In [3]:
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."


From above we can observe that the data for news is delivered for weekdays only, which corresponds to the only available DJIA price data.

In [4]:
df1 = pd.read_csv(r"C:\Users\Dennis\Desktop\Coding\Final Project\Data\upload_DJIA_table.csv")

In [5]:
df['Label'].value_counts()

1    1065
0     924
Name: Label, dtype: int64

In [6]:
df.Label.value_counts(normalize=True)

1    0.535445
0    0.464555
Name: Label, dtype: float64

In [7]:
# Data is balanced and aligned (dates correspond)

In [8]:
#df = data[data['Date'] < '2015-01-01']
#test = data[data['Date'] > '2014-12-31']
#y_train = df.Label.values
#y_test = test.Label.values

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
Date     1989 non-null object
Label    1989 non-null object
Top1     1989 non-null object
Top2     1989 non-null object
Top3     1989 non-null object
Top4     1989 non-null object
Top5     1989 non-null object
Top6     1989 non-null object
Top7     1989 non-null object
Top8     1989 non-null object
Top9     1989 non-null object
Top10    1989 non-null object
Top11    1989 non-null object
Top12    1989 non-null object
Top13    1989 non-null object
Top14    1989 non-null object
Top15    1989 non-null object
Top16    1989 non-null object
Top17    1989 non-null object
Top18    1989 non-null object
Top19    1989 non-null object
Top20    1989 non-null object
Top21    1989 non-null object
Top22    1989 non-null object
Top23    1988 non-null object
Top24    1986 non-null object
Top25    1986 non-null object
dtypes: object(27)
memory usage: 419.7+ KB


In [10]:
#print('The size of the training set is {} news, the size of the test set is {} news'.format(len(df), len(test)))

In [11]:
# Check if there are NaNs
df.isnull().values.any()

True

In [12]:
# Take a look at NaNs values
df[df.isnull().any(axis=1)]

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
277,2009-09-15,1,b'The Church of Scientology won\'t be dissolve...,b'New virus from rats can kill 80 per cent of ...,b'The gruesome spectacle of dolphins being sla...,b'The End of Innocence in Afghanistan: \'The G...,b'France approves Internet piracy bill',b'The Rural Doctors Association says right now...,b'Al Jazeera English - Africa - Shabab to aven...,"b""How Sri Lanka governs through detentions - S...",...,b'In an equine echo of the controversy surroun...,b'UPDATE: 5-New York homes raided in terrorism...,b'Population Growth Impeding Progress on the M...,b'Global Population to Reach 7 Billion by 2011',b'Government Funded Feminist Porn ',b'Can someone enlighten me re:Holy Land disput...,b'Human Rights Watch official suspended for co...,NaN,NaN,NaN
348,2009-12-24,1,b'Woman knocks down Pope Benedict at Christmas...,b'Ugandan President Museveni says he will bloc...,"b""Venezuela's Chavez threatens to kick out car...",b'Woman who knocked down pope had pasta in her...,"b'450 people from 21 countries, including 30 f...",b'Chvez declares Angel Falls is no more: World...,b'Drug tests catch out 10 police (UK)',"b'Once the worlds fourth-largest lake, the Ara...",...,b'Pig farts spark Australia gas scare - I shit...,b'Bin Laden daughter flees to Saudi embassy in...,b'Traumatic Brain Injuries: Growing evidence ...,b'The Pakistan Supreme Court has ordered the P...,"b""Here's a video the Taliban released this mor...",b'Fireworks set off aboard airliner',"b'Five VA men may face terrorism charges, Paki...","b""Ayatollah Montazeri's Legacy: In death he m...",NaN,NaN
681,2011-04-21,1,"Director of ""Restrepo"" and Photographer Chris ...",Everyone within 20km of Fukushima will be forc...,Bahrain: Activist Zaynab Al-Khawaja's letter t...,"More than 80,000 people are claiming incapacit...",Libya: 'mission creep' claims as UK sends in m...,Japan considers banning access to evacuation zone,Chinese oil giant Sinopec has stopped exportin...,6.0-magnitude earthquake strikes off east coas...,...,Russian migration official fired in racism row...,Small amounts of radioactive iodine found in b...,"War photographer, Oscar-nominated film directo...",Effeminate boys are being sent to a special ca...,Japan Government Declares 12-Mile Area Around ...,TEPCO admitted Wednesday that nuclear fuel in ...,Fukushima evacuees face arrest if they return ...,Prince Charles wins some kind of a record,NaN,NaN


In [13]:
# Replace NaN-Values with whitespace
df = df.replace(np.nan, ' ', regex=True)

In [14]:
# Check if there are NaNs
df.isnull().values.any()

False

In [15]:
df.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [16]:
# Dra
df['Date'] = pd.to_datetime(df['Date'])
df1['Date'] = pd.to_datetime(df1['Date'])

In [17]:
df['Label'] = pd.to_numeric(df['Label'])

In [18]:
df1 = df1.rename(columns={"Adj Close": "AdjClose"})

In [19]:
df1 = df1.sort_values(by=['Date'])

In [20]:
df = pd.merge(df, df1)

In [21]:
df['pct'] = (df['Close'] - df['Open']) / df['Open'] * 100

In [22]:
df['Delta']=df['AdjClose']-df['Open']

In [23]:
# Compute the logarithmic returns using the Closing price 
df['Log_Ret'] = np.log(df['Close'] / df['Close'].shift(1))

# The rolling function uses a window of 252 trading days. 
# Each of the days in the selected lookback period is assigned an equal weight. 
# The user can choose a longer or a shorter period as per his need.

# Compute Volatility using the pandas rolling standard deviation function
df['Volatility'] = df['Log_Ret'].rolling(window=252).std() * np.sqrt(252)

### Price Volatility (High and  Low difference)

In [24]:
df['Delta_Vol'] = df['High'] - df['Low']

In [25]:
df['Delta_Vol'].median()

141.06933600000048

The historical median fo Delta Volatility over the previous 20 years (1896-05-27 to 2020-04-24) was 94.3; and mean 121.3.

In [26]:
def to_label (row):
    if row['Delta_Vol'] < 121.3:
        return 0
    if row['Delta_Vol'] > 121.3:
        return 1

In [27]:
df['Vol_Label'] = df.apply (lambda row: to_label(row), axis=1)

In [28]:
count_zero= df.loc[df['Vol_Label'] == 0]
number_of_zeros= len(count_zero)

count_one = df.loc[df['Vol_Label'] == 1]
number_of_ones = len(count_one)

#print(' Total amount of values/rows:' , count,"\n",'Total number of days DJIA Delta Volatility was above its historical average: ', 
 #     number_of_ones, "\n", "Total number of days DJIA Delta Volatility was below its historical average:", number_of_zeros)

In [29]:
df.sample(5)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Low,Close,Volume,AdjClose,pct,Delta,Log_Ret,Volatility,Delta_Vol,Vol_Label
1257,2013-08-07,0,Stephen Fry writes an open letter to Cameron a...,"""Some girls have been married 60 times by the ...",41yo man walks free after admitting having sex...,Wikipedia Boosts Security after XKeyscore Reve...,"McDonald's burgers ""unfit for human consumption""",The Taliban ambushed a convoy of a female Afgh...,The Magical World Where McDonald's Pays $15 an...,"Python That Killed Boys Was House Pet: ""The py...",...,15421.750000,15470.669922,88430000,15470.669922,-0.293500,-45.540039,-0.003102,0.105929,94.459961,0
970,2012-06-14,1,Pure ecstacy (MDMA) can be safe for adults; sh...,Canadian researchers thwart Ebola virus - The ...,Germany bans ultraconservative Islamic organiz...,"Over 50 Zetas Drug Cartel Members, Including 2...",Italian prosecutors suspect Vatican bank is la...,Child shelter home scandal rocks India: '5-yr-...,Cleric declares jihad against polio campaign i...,Gaza water too contaminated to drink,...,12497.660156,12651.910156,128640000,12651.910156,1.232372,154.020508,0.012369,0.210473,201.019532,1
324,2009-11-19,0,b'FIFA and Germany helped finance a soccer sta...,b'Depressed woman loses benefits over Facebook...,b'Illegal Israeli settlements are diverting wa...,b'New poll: 53% of Israelis think ethnic clean...,b'British TV documentary on Israel lobby is ac...,"b'Regarding Afghanistan: ""In 1981, we gave the...","b""How the British lied to put a killer in powe...",b'50 oil tankers loitering off British coast a...,...,10256.110352,10332.440430,196250000,10332.440430,-0.891000,-92.889648,-0.009044,0.288466,169.219726,1
1564,2014-10-24,1,U.S. threatens sanctions on buyers of ISIS oil,Canada attack 'not linked to IS',Poll: 74 percent of Denmark's citizens want to...,World's Longest Snake Has Virgin Birth1st Reco...,Lebanese MP Nicolas Fattoush repeatedly punche...,"""Since church ministers declared Ebola was a p...",Sixty more women and girls reported kidnapped ...,Canada to toughen terror laws,...,16649.720703,16805.410156,90400000,16805.410156,0.769748,128.371094,0.007616,0.104772,161.990235,1
1848,2015-12-10,1,"""The US State Department has approved a $1.29 ...",Trump cancels trip to Israel after Netanyahu r...,Mina stampede deaths three times higher than a...,A Canadian ISP has received a public backlash ...,Nicola Sturgeon removes Donald Trump as busine...,Kim Jong-uns uncle admits he defected to U.S. ...,Two detained in Finland over ISIS executions -...,Fukishima Radiation Off West Coast of North Am...,...,17474.660156,17574.750000,107310000,17574.750000,0.466354,81.580078,0.004702,0.153514,223.080078,1


# Stochastic %K

In [30]:
def StoK(close, low, high, n):  
    StoK = ((close - low.rolling(n).min() / high.rolling(n).max() - low.rolling(n).min())) * 100
    return StoK

In [31]:
df['Stoch_%K'] = StoK(df['Close'], df['Low'], df['High'], 14)

### Stochastic %D

In [32]:
def StoD(close, low, high, n):
    StoK = ((close - low.rolling(n).min() / high.rolling(n).max() - low.rolling(n).min())) * 100
    StoD = StoK.rolling(window = 3).mean()
    return StoD

In [33]:
df['Stoch_%D'] = StoD(df['Close'], df['Low'], df['High'], 10)

### Momentum

Momentum is the speed or velocity of price changes in a stock, security, or tradable instrument. Momentum shows the rate of change in price movement over a period of time to help investors determine the strength of a trend. Stocks that tend to move with the strength of momentum are called momentum stocks.

[Source](https://www.investopedia.com/articles/technical/081501.asp)

In [34]:
df['Momentum'] = df['Close'].diff(periods=3)

### Rate of Change

In [35]:
df['Rate_of_Change'] = df['Close'].pct_change()

### William's %R

In [36]:
df['Williams_R'] = df.ta.willr(df['High'], df['Low'], df['Close'], lbp=14, fillna=False)

# >>> tranform to float 64

In [37]:
df['Williams_R'] = pd.to_numeric(df['Williams_R'])

### A/D Oscillator

In [38]:
df['A/D_Oscillator'] = df.ta.adosc(df['High'], df['Low'], df['Close'], df['Volume'], df['Open'])

### Disparity 5

In [39]:
df['MA5'] = df.Close.rolling(window=5).mean()
df['Disparity_5'] = df['Close']/df['MA5']*100

### Moving Averages

In [40]:
df['MA20'] = df.AdjClose.rolling(window=20).mean()
df['MA60'] = df.AdjClose.rolling(window=60).mean()
df['MA150'] = df.AdjClose.rolling(window=150).mean()

### RSI

In [41]:
df['rsi'] = ta.rsi(df.AdjClose, 14)

### MACD

In [42]:
df['26 ema'] = df.ta.ema(df['Close'], length = 26)
df['12 ema'] = df.ta.ema(df['Close'], length = 12)
df['MACD'] = (df['12 ema'] - df['26 ema'])

## Data Cleaning and Preparation

In [43]:
df1 = df1.sort_values(by=['Date'])

In [44]:
df = pd.merge(df, df1)

In [45]:
df['AllNews'] = df.iloc[:,2:27].astype(str).apply(' '.join, axis=1)

In [46]:
df.insert(2,'AllNews', df.pop("AllNews"))

In [47]:
df.head(1)

,Date,Label,AllNews,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,A/D_Oscillator,MA5,Disparity_5,MA20,MA60,MA150,rsi,26 ema,12 ema,MACD
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...","b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.columns

Index(['Date', 'Label', 'AllNews', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5',
       'Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13',
       'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21',
       'Top22', 'Top23', 'Top24', 'Top25', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjClose', 'pct', 'Delta', 'Log_Ret', 'Volatility',
       'Delta_Vol', 'Vol_Label', 'Stoch_%K', 'Stoch_%D', 'Momentum',
       'Rate_of_Change', 'Williams_R', 'A/D_Oscillator', 'MA5', 'Disparity_5',
       'MA20', 'MA60', 'MA150', 'rsi', '26 ema', '12 ema', 'MACD'],
      dtype='object')

In [49]:
# !Change the last number (to 39?) for dropping all the unnecessary columns
df.drop(df.iloc[:,3:39], axis=1, inplace=True)

In [50]:
df.columns

Index(['Date', 'Label', 'AllNews', 'Vol_Label', 'Stoch_%K', 'Stoch_%D',
       'Momentum', 'Rate_of_Change', 'Williams_R', 'A/D_Oscillator', 'MA5',
       'Disparity_5', 'MA20', 'MA60', 'MA150', 'rsi', '26 ema', '12 ema',
       'MACD'],
      dtype='object')

In [51]:
df = df[['Date', 'Label', 'Vol_Label', 'AllNews', 'Stoch_%K',
       'Stoch_%D', 'Momentum', 'Rate_of_Change', 'Williams_R',
       'A/D_Oscillator', 'MA5', 'Disparity_5', 'MA20', 'MA60', 'MA150', 'rsi',
       '26 ema', '12 ema', 'MACD']]

In [52]:
df.head(3)

,Date,Label,Vol_Label,AllNews,Stoch_%K,Stoch_%D,Momentum,Rate_of_Change,Williams_R,A/D_Oscillator,MA5,Disparity_5,MA20,MA60,MA150,rsi,26 ema,12 ema,MACD
0,2008-08-08,0,1,"b""Georgia 'downs two Russian warplanes' as cou...",NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-08-11,1,1,b'Why wont America and Nato help us? If they w...,NaN,NaN,NaN,0.004093,0.0,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN
2,2008-08-12,0,1,b'Remember that adorable 9-year-old who sang a...,NaN,NaN,NaN,-0.011872,0.0,NaN,NaN,NaN,NaN,NaN,NaN,82.779565,NaN,NaN,NaN


In [53]:
df.dropna(inplace= True)

In [54]:
df.isnull().sum().sum()

0

# Sentiment analysis

In [55]:
SentAn = SentimentIntensityAnalyzer()

In [56]:
def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [57]:
import time
start = time.time()

start_vect=time.time()
print("Sentiment Analysis initialized...")

#df['AllNews'] = df['AllNews'].astype(str) # Make sure data is treated as a string
df['AllNews'+'_pos']= df['AllNews'].apply(lambda x:SentAn.polarity_scores(x)['pos'])
df['AllNews'+'_neg']= df['AllNews'].apply(lambda x:SentAn.polarity_scores(x)['neg'])
df['AllNews'+'_comp']= df['AllNews'].apply(lambda x:SentAn.polarity_scores(x)['compound'])
df['AllNews'+'_sub'] = df['AllNews'].apply(detect_subjectivity)
    
print("Vaderization process completed after %0.2f Minutes"%((time.time() - start_vect)/60))

Sentiment Analysis initialized...
Vaderization process completed after 0.48 Minutes


In [58]:
df.head(3)

,Date,Label,Vol_Label,AllNews,Stoch_%K,Stoch_%D,Momentum,Rate_of_Change,Williams_R,A/D_Oscillator,...,MA60,MA150,rsi,26 ema,12 ema,MACD,AllNews_pos,AllNews_neg,AllNews_comp,AllNews_sub
149,2009-03-13,1,1,"b""What will Americans do when it's not an Arab...",75315.605068,63730.760795,297.489746,0.007520,-19.354219,2.086435e+08,...,7953.539779,9096.661172,47.437094,7271.503339,7015.225102,-256.278238,0.055,0.154,-0.9898,0.350456
150,2009-03-16,0,1,b'Hello Wikileaks? Huge UK Bank gags newspape...,74614.628568,73283.067517,286.570313,-0.000970,-20.103934,2.348707e+08,...,7925.086955,9066.545505,47.271738,7267.463848,7046.262811,-221.201037,0.042,0.144,-0.9929,0.396197
151,2009-03-17,1,1,b'Australian Government adds Wikileaks to bann...,92487.595526,80805.221268,225.640136,0.024765,-0.708939,3.228300e+08,...,7901.276294,9037.301175,51.856322,7276.962837,7100.022409,-176.940428,0.045,0.220,-0.9978,0.283046


# Pre-Processing

In [59]:
# in order to predict the next day's market movement / volatility, we shift our Label and Vol_Lablel columns for one row:

df.Label = df.Label.shift(-1)
df.Vol_Label = df.Vol_Label.shift(-1)

In [60]:
df.dropna(inplace= True)

In [61]:
df.head(2)

,Date,Label,Vol_Label,AllNews,Stoch_%K,Stoch_%D,Momentum,Rate_of_Change,Williams_R,A/D_Oscillator,...,MA60,MA150,rsi,26 ema,12 ema,MACD,AllNews_pos,AllNews_neg,AllNews_comp,AllNews_sub
149,2009-03-13,0.0,1.0,"b""What will Americans do when it's not an Arab...",75315.605068,63730.760795,297.489746,0.00752,-19.354219,2.086435e+08,...,7953.539779,9096.661172,47.437094,7271.503339,7015.225102,-256.278238,0.055,0.154,-0.9898,0.350456
150,2009-03-16,1.0,1.0,b'Hello Wikileaks? Huge UK Bank gags newspape...,74614.628568,73283.067517,286.570313,-0.00097,-20.103934,2.348707e+08,...,7925.086955,9066.545505,47.271738,7267.463848,7046.262811,-221.201037,0.042,0.144,-0.9929,0.396197


In [62]:
from nltk.tokenize import word_tokenize

df['AllNews'] = [word_tokenize(x) for x in df['AllNews']]

In [63]:
df.sample(1)

,Date,Label,Vol_Label,AllNews,Stoch_%K,Stoch_%D,Momentum,Rate_of_Change,Williams_R,A/D_Oscillator,...,MA60,MA150,rsi,26 ema,12 ema,MACD,AllNews_pos,AllNews_neg,AllNews_comp,AllNews_sub
1983,2016-06-24,0.0,1.0,"[David, Cameron, to, Resign, as, PM, After, EU...",4344.677124,29694.279908,-428.980469,-0.033886,-93.267722,-6.854198e+07,...,17778.381087,17290.658027,39.370487,17760.611799,17751.397004,-9.214795,0.051,0.093,-0.8881,0.37551


In [64]:
stop = stopwords.words('english')
# 
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))
# 
def isSymbol(inputString):
    return bool(re.match(r'[^\w]', inputString))
# lemma
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def check(word):
    """
         If you need this word, then True
         If it should be removed, False
    """
    word= word.lower()
    if word in stop:
        return False
    elif hasNumbers(word) or isSymbol(word):
        return False
    else:
        return True

# Combine the above methods
def preprocessing(sen):
    res = []
    for word in sen:
        if check(word):
            # The use of this paragraph is just to remove the mark left in the Python byte store str. . The data was not processed before, and this will not happen in other cases.
            word = word.lower().replace("b'", '').replace('b"', '').replace('"', '').replace("'", '').replace("b", '')
            res.append(wordnet_lemmatizer.lemmatize(word))   
    return res

In [65]:
df['AllNews'] = [preprocessing(x) for x in df['AllNews']]

# Text Exploration

In [66]:
non_decrease = df[df['Label']==1]
decrease = df[df['Label']==0]

In [67]:
non_decrease_word=[]
decrease_word=[]
for each in df['AllNews']:
    non_decrease_word.append(each)

for each in df['AllNews']:
    decrease_word.append(each)

# Text as Feature

In [68]:
path = get_tmpfile("word2vec.model")

In [69]:
model = Word2Vec(df['AllNews'], size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [70]:
model.train(df.AllNews, total_examples=1, epochs=1)

(533392, 540577)

In [71]:
def get_SenVector(word_list):
    res = np.zeros([100])
    for word in word_list:
        if word in model:
            res += model[word]
    return res

In [72]:
df['AllNews'] = [get_SenVector(x) for x in df['AllNews']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning:

Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



# Model Estimation

In [73]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1839 entries, 149 to 1987
Data columns (total 23 columns):
Date              1839 non-null datetime64[ns]
Label             1839 non-null float64
Vol_Label         1839 non-null float64
AllNews           1839 non-null object
Stoch_%K          1839 non-null float64
Stoch_%D          1839 non-null float64
Momentum          1839 non-null float64
Rate_of_Change    1839 non-null float64
Williams_R        1839 non-null float64
A/D_Oscillator    1839 non-null float64
MA5               1839 non-null float64
Disparity_5       1839 non-null float64
MA20              1839 non-null float64
MA60              1839 non-null float64
MA150             1839 non-null float64
rsi               1839 non-null float64
26 ema            1839 non-null float64
12 ema            1839 non-null float64
MACD              1839 non-null float64
AllNews_pos       1839 non-null float64
AllNews_neg       1839 non-null float64
AllNews_comp      1839 non-null float64
AllNe

## Technical indicators

In [75]:
df.columns

Index(['Date', 'Label', 'Vol_Label', 'AllNews', 'Stoch_%K', 'Stoch_%D',
       'Momentum', 'Rate_of_Change', 'Williams_R', 'A/D_Oscillator', 'MA5',
       'Disparity_5', 'MA20', 'MA60', 'MA150', 'rsi', '26 ema', '12 ema',
       'MACD', 'AllNews_pos', 'AllNews_neg', 'AllNews_comp', 'AllNews_sub'],
      dtype='object')

In [76]:
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']

In [77]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble

## Logistic Regression

### Market movement

In [78]:
Logis = LogisticRegression()
Model1 = Logis.fit(train.iloc[:,4:19],train['Label'])
Predicted = Model1.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Label"],Predicted))

Accuracy: 0.5092838196286472


### Volatility

In [79]:
Model2 = Logis.fit(train.iloc[:,4:19],train['Vol_Label'])
Predicted = Model2.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Vol_Label"],Predicted))

Accuracy: 0.7718832891246684


## Random Forest

### Market

In [81]:
rfc = RandomForestClassifier()
Model3 = rfc.fit(train.iloc[:,4:19],train['Label'])
Predicted = Model3.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Label"],Predicted))

Accuracy: 0.5305039787798409


### Volatility

In [82]:
Model4 = rfc.fit(train.iloc[:,4:19],train['Vol_Label'])
Predicted = Model4.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Vol_Label"],Predicted))

Accuracy: 0.7453580901856764


## GradientBoost

In [83]:
# Fit the model
gbc = ensemble.GradientBoostingClassifier(n_estimators=500, max_depth=2, loss='deviance')
gbc.fit(train.iloc[:,4:19],train['Label'])
# Make predictions.
Predicted = gbc.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Label"],Predicted))

Accuracy: 0.53315649867374


In [84]:
# Fit the model
gbc = ensemble.GradientBoostingClassifier(n_estimators=500, max_depth=2, loss='deviance')
gbc.fit(train.iloc[:,4:19],train['Vol_Label'])
# Make predictions.
Predicted = gbc.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Vol_Label"],Predicted))

Accuracy: 0.7374005305039788


## XGBoost

In [85]:
xgb = XGBClassifier(booster='gblinear', feature_selector='shuffle', objective='reg:squarederror')
xgb.fit(train.iloc[:,4:19],train['Label'])
Predicted = xgb.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Label"],Predicted))

Accuracy: 0.506631299734748


In [86]:
xgb = XGBClassifier(booster='gblinear', feature_selector='shuffle', objective='reg:squarederror')
xgb.fit(train.iloc[:,4:19],train['Vol_Label'])
Predicted = xgb.predict(test.iloc[:,4:19])
print("Accuracy:",accuracy_score(test["Vol_Label"],Predicted))

Accuracy: 0.6896551724137931
